# First-order languages and Set Theory

## First-Order languages

A first-order language is determined by a vocabulary consisting of _variables_, _constants_, _functions_ and _relations_. First, we encode functions and relations.

In [1]:
case class Function(name: String, degree: Int)

case class Relation(name: String, degree: Int)

defined class Function
defined class Relation

We form two kinds of _expressions_, _terms_ and _formulas_ using these. Both of these are defined recursively.

In [2]:
sealed trait Expression

sealed trait Term extends Expression

sealed trait Formula extends Expression

object Term{
    case class Var(name: String) extends Term
    
    case class Const(name: String) extends Term 

    case class Recursive(function: Function, arguments: Vector[Term]) extends Term{
        require(function.degree == arguments.length)
    }
}

object Formula{
    case class Equality(lhs: Term, rhs: Term) extends Formula
    
    case class Atomic(relation: Relation, arguments: Vector[Term]) extends Formula{
        require(relation.degree == arguments.length)
    }

    case class Or(p: Formula, q: Formula) extends Formula

    case class And(p: Formula, q: Formula) extends Formula

    case class Implies(p: Formula, q: Formula) extends Formula

    case class Equivalent(p: Formula, q: Formula) extends Formula

    case class Not(p: Formula) extends Formula

    case class ForAll(x: Term.Var, p: Formula) extends Formula

    case class Exists(x: Term.Var, p: Formula) extends Formula

}

defined trait Expression
defined trait Term
defined trait Formula
defined object Term
defined object Formula

In [4]:
val sum = Function("+", 2)
val succ = Function("succ", 1)
// Some valid terms
import Term._
val n = Var("n")
val m = Var("m")
val one = Const("1")
Recursive(sum, Vector(n, m))
Recursive(succ, Vector(n))
val two = Recursive(succ, Vector(one))

sum: Function = Function("+", 2)
succ: Function = Function("succ", 1)
import Term._

n: Var = Var("n")
m: Var = Var("m")
one: Const = Const("1")
res3_6: Recursive = Recursive(Function("+", 2), Vector(Var("n"), Var("m")))
res3_7: Recursive = Recursive(Function("succ", 1), Vector(Var("n")))
two: Recursive = Recursive(Function("succ", 1), Vector(Const("1")))

In [6]:
// An invalid term
import scala.util.Try
val wrong = Try(Recursive(succ, Vector(one, n)))

import scala.util.Try

wrong: Try[Recursive] = Failure(
  java.lang.IllegalArgumentException: requirement failed
)

In [9]:
// Some formulas
import Formula._
val f1 = Equality(one, two)
val f2 = Equality(n, one)
val leq = Relation("<=", 2)
val f3 = Atomic(leq, Vector(one, two))
val f4 = ForAll(n, f2)
val f5 = Exists(n, f2)
val f6 = ForAll(m, f1)

import Formula._

f1: Equality = Equality(
  Const("1"),
  Recursive(Function("succ", 1), Vector(Const("1")))
)
f2: Equality = Equality(Var("n"), Const("1"))
leq: Relation = Relation("<=", 2)
f3: Atomic = Atomic(
  Relation("<=", 2),
  Vector(Const("1"), Recursive(Function("succ", 1), Vector(Const("1"))))
)
f4: ForAll = ForAll(Var("n"), Equality(Var("n"), Const("1")))
f5: Exists = Exists(Var("n"), Equality(Var("n"), Const("1")))
f6: ForAll = ForAll(
  Var("m"),
  Equality(Const("1"), Recursive(Function("succ", 1), Vector(Const("1"))))
)

The formulas in mathematical notation are

* $1 = 2$
* $n = 1$
* $1 \leq 2$
* $\forall n\ n = 1$
* $\exists n\ n = 1$
* $\exists m\ 1 = 2$

## The language of Sets

The language of sets has:

* a single constant $\phi$
* a single relation $\in$ of degree $2$.

We introduce these as well as a convenience method.

In [10]:
import Term._, Formula._

val phi = Const("empty-set")
val in = Relation("in", 2)

def belongs(x: Term, y: Term) = Formula.Atomic(in, Vector(x, y))

import Term._, Formula._


phi: Const = Const("empty-set")
in: Relation = Relation("in", 2)
defined function belongs

As an example, we define extensionality, i.e. if two sets have the same members, they are equal.
$$\forall x\forall y((\forall a\ (a\in x\iff a \in y)) \implies (x = y))$$

In [11]:
val a = Var("a")
def sameMembers(p: Term, q: Term): Formula = 
    ForAll(a, Equivalent(belongs(a, p), belongs(a, q)))

a: Var = Var("a")
defined function sameMembers

In [12]:
val x = Var("x")
val y = Var("y")
val extensionality = 
   ForAll(x, 
         ForAll(y,
               Implies(sameMembers(x, y), Equality(x, y))))
 

x: Var = Var("x")
y: Var = Var("y")
extensionality: ForAll = ForAll(
  Var("x"),
  ForAll(
    Var("y"),
    Implies(
      ForAll(
        Var("a"),
        Equivalent(
          Atomic(Relation("in", 2), Vector(Var("a"), Var("x"))),
          Atomic(Relation("in", 2), Vector(Var("a"), Var("y")))
        )
      ),
      Equality(Var("x"), Var("y"))
    )
  )
)

## Variables and free variables

* The _value_ of a term can depend on some variables.
* The _truth value_ of a formula can depend on some variables.
* These are called _free variables_.

### Examples

In the language of natural numbers

* $1 = 2$; no free variables
* $n = 1$; free variable n
* $1 \leq 2$; no free variables
* $\forall n\ n = 1$; no free variables
* $\exists n\ n +5 < m$; free variable m
* $\exists m\ 1 = 2$; no free variables



In [18]:
// Variable in a term
def variables(t: Term): Set[Var] = t match {
        case Const(name) => Set()
        case Recursive(function, arguments) =>
            for{
                arg : Term <- arguments.toSet
                x <- variables(arg)
            } yield x
        case Var(name) => Set(Var(name))
    }

defined function variables

In [19]:
variables(Recursive(sum, Vector(n, m)))

res18: Set[Var] = Set(Var("n"), Var("m"))

In [20]:
val p = Recursive(succ, Vector(m))
variables(Recursive(sum, Vector(n, p)))

p: Recursive = Recursive(Function("succ", 1), Vector(Var("m")))
res19_1: Set[Var] = Set(Var("n"), Var("m"))

In [21]:
variables(p)

res20: Set[Var] = Set(Var("m"))

In [22]:
def freeVariables(fmla: Formula) : Set[Var] = fmla match {
    case Equality(lhs, rhs) => 
        variables(lhs) union variables(rhs)
    case And(p, q) => 
        freeVariables(p) union freeVariables(q)
    case Implies(p, q) => 
        freeVariables(p) union freeVariables(q)
    case Equivalent(p, q) => 
        freeVariables(p) union freeVariables(q)
    case Not(p) => 
        freeVariables(p)
    case ForAll(x, p) => 
        freeVariables(p) - x
    case Atomic(relation, arguments) =>
        for {
            arg: Term <- arguments.toSet
            x <- variables(arg)
        } yield x
    case Exists(x, p) => 
        freeVariables(p) - x 
    case Or(p, q) => 
        freeVariables(p) union freeVariables(q)
    }

defined function freeVariables

In [23]:
val fmla1 = Exists(x, belongs(x, y))
val fmla2 = ForAll(y, fmla1)
val fmla3 = belongs(x, y)

fmla1: Exists = Exists(
  Var("x"),
  Atomic(Relation("in", 2), Vector(Var("x"), Var("y")))
)
fmla2: ForAll = ForAll(
  Var("y"),
  Exists(Var("x"), Atomic(Relation("in", 2), Vector(Var("x"), Var("y"))))
)
fmla3: Atomic = Atomic(Relation("in", 2), Vector(Var("x"), Var("y")))

In [24]:
freeVariables(fmla1)
freeVariables(fmla2)
freeVariables(fmla3)

res23_0: Set[Var] = Set(Var("y"))
res23_1: Set[Var] = Set()
res23_2: Set[Var] = Set(Var("x"), Var("y"))